In [109]:
#A brute force implementation of Algorithm1

In [144]:
import pandas as pd
import numpy as np
from random import sample,seed
import heapq

In [145]:
LAMB = 0.01

In [146]:
df = pd.DataFrame(pd.read_csv('../data/compas-binary.csv'))
df

seed(10)

idx = sample(range(df.shape[0]),df.shape[0])

In [147]:
x = df.as_matrix()[:,:13]

y = df.as_matrix()[:,13]

In [148]:
#support
supp = [(x[:,i]*y).mean() for i in range(13)]
supp

[0.07050818010713769,
 0.05993919212393224,
 0.049804546112639354,
 0.06906037353409585,
 0.2265817286810482,
 0.057333140292456924,
 0.4327493846822065,
 0.420587809468655,
 0.36557115969306503,
 0.09497611119154481,
 0.08817142029824815,
 0.07137686405096279,
 0.20819458520341683]

In [149]:
#confidence
conf1 = [sum(x[:,i]*y)/sum(x[:,i]) for i in range(13)]
conf1

[0.3667168674698795,
 0.592274678111588,
 0.5300462249614792,
 0.5184782608695652,
 0.45773618016964024,
 0.32459016393442625,
 0.45069360675512665,
 0.44644229291532195,
 0.4233735747820255,
 0.4932330827067669,
 0.28986197049024276,
 0.37864823348694315,
 0.6614535418583257]

In [150]:
#confidence
conf2 = [sum((x[:,i]==0)*y)/sum((x[:,i]==0)) for i in range(13)]
conf2

[0.48557089084065247,
 0.4481314432989691,
 0.45573665707893896,
 0.45415065976281943,
 0.4676032110091743,
 0.4923509759099701,
 0.7527272727272727,
 0.7275,
 0.711558854718982,
 0.45544199390353235,
 0.5382854764877236,
 0.4822479928635147,
 0.37143460807099093]

In [151]:
#x_idx = [conf1[i]>0.5 or conf2[i]>0.5 for i in range(len(conf1))]
x_idx = [conf1[i]>0.5 for i in range(len(conf1))]
x_idx[0] = True
x_idx

[True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True]

In [152]:
x = x[:,x_idx]

In [153]:
nrule = x.shape[1]
ndata = len(y)

In [154]:
# calculate the bound with dp=all leaves except leaf l
def bound(tree, x, y, l, lamb = LAMB):
    return (sum(tree.get_ncc()[:l]+tree.get_ncc()[l+1:]))/len(y) + lamb*len(tree.get_prefix())

# calculate the risk
#def Risk(tree, x, y, lamb = LAMB):
#    return 1-sum(tree.get_ncc())/len(y) + + lamb*tree.get_depth()

In [155]:
# z is the vector defined in algorithm 5
z = pd.DataFrame([-1]*ndata).as_matrix()
for i in range(ndata):
    if z[i] == -1:
        tag1 = np.array([True]*ndata)
        for j in range(nrule):
            rule_label = x[i][j]
            #print("prefix:",prefix)
            #print("rule_index:",rule_index)
            #print("x:",x)
            tag1 = (x[:,j] == rule_label)*tag1
            
        y_l = y[tag1]
        pred = int(y_l.sum()/len(y_l) > 0.5)
        tag2 = (y_l != pred)
        z[tag1,0] = tag2

tag1.dot(z)[0]

0

In [156]:
def calcul(prefix, x, y):
    """
    Function for calculating the predictions, number of data captured,
    and number of data correctly captured by the leaves.
    """
    prediction = []
    num_captured = []
    num_captured_incorrect = []
    B0 = [] # b0 is defined in (28)
    for i in range(len(prefix)):
        tag = np.array([True]*ndata)
        for j in range(len(prefix[i])):
            rule_index = abs(prefix[i][j])-1
            rule_label = int(prefix[i][j]>0)
            #print("prefix:",prefix)
            #print("rule_index:",rule_index)
            #print("x:",x)
            tag = (x[:,rule_index] == rule_label)*tag
            
        # the y's of these data captured by leaf i
        y_leaf = y[tag]
        
        #b0 = sum([tag[k]*z[k] for k in range(ndata)])/ndata
        b0 = tag.dot(z)[0]/ndata
        B0.append(b0)
        
        num_cap = len(y_leaf)
        num_captured.append(num_cap)
        
        if len(y_leaf)>0:
            pred = int(y_leaf.sum()/len(y_leaf) > 0.5)
            prediction.append(pred)
            num_cap_incor = sum(y_leaf != pred)
            num_captured_incorrect.append(num_cap_incor)
        else:
            prediction.append(0)
            num_captured_incorrect.append(0)
        
    return prediction, num_captured, num_captured_incorrect, B0

In [157]:
[i for i in range(10) if i==3]

[3]

In [171]:
class CacheTree:
    """
    A tree data structure.
    prefix: a 2-d tuple to record the prefixes of leaves
    prediction: a list to record the predictions of leaves
    num_captured: a list to record number of data captured by the leaves
    num_captured_correct: a list to record number of data correctly captured by the leaves
    """
    def __init__(self, x, y, prefix, 
                 prediction=None,
                 num_captured=None,
                 num_captured_incorrect=None,
                 deadleaf = None,
                 splitleaf = None,
                 lbound=None,lamb=LAMB,tag = False, curiosity = None, B0 = None):
        self.prefix = prefix
        self.prediction = prediction
        self.num_captured = num_captured
        self.num_captured_incorrect = num_captured_incorrect
        #self.split = split #which leaf to split
        self.deadleaf = deadleaf #a list indicate which leaves will never be split (support bound)
        self.splitleaf = splitleaf #a list indicate which leaves will be split
        self.lbound = lbound #a list of lower bound
        self.curiosity = curiosity
        self.B0 = B0
        #self.tag = tag
        #self.depth = max([len(l) for l in self.prefix])
        
        ndata = len(y)
        l = len(prefix)
        if prediction==None:
            self.prediction, self.num_captured, self.num_captured_incorrect, self.B0 = calcul(self.prefix, x, y)
            self.deadleaf = [0]*l
            self.splitleaf = [[1]*l]
            self.lbound = [sum(self.num_captured_incorrect[:i]+self.num_captured_incorrect[i+1:])/ndata + lamb*l 
                           for i in range(l)]
        
        self.curiosity = 1
        
        ###min bound
        #self.curiosity = min([self.lbound[i]
        #                      for i in range(l) if self.splitleaf[0][i]==1])
        
        ###min curiosity
        #self.curiosity = min([self.lbound[i]/((ndata-self.num_captured[i])/len(y)) 
        #                      for i in range(l) if self.splitleaf[0][i]==1])
            
    def get_prefix(self):
        return self.prefix
    
    def get_pred(self):
        return self.prediction
    
    def get_cap(self):
        return self.num_captured
    
    def get_ncc(self):
        return self.num_captured_incorrect
    
    def get_deadleaf(self):
        return self.deadleaf
    
    def get_splitleaf(self):
        return self.splitleaf
    
    def set_deadleaf(self,i):
        self.deadleaf[i] = 1
        return
    
    def set_not_splitleaf(self,i):
        self.splitleaf[i] = 0
        return
        
    def get_lbound(self):
        return self.lbound
    
    #def get_tag(self):
    #    return self.tag
    
    #def get_depth(self):
    #    return self.depth
    
    def get_curiosity(self):
        return self.curiosity
    
    def get_B0(self):
        return self.B0
    
    def __lt__(self, other):
        return self.curiosity<other.curiosity

In [159]:
from gmpy2 import mpz
from gmpy2 import mpfr
import gmpy2

In [160]:
gmpy2.get_context().precision=100

In [161]:
aaa=mpfr('2')
bbb=mpfr('0.3')

In [162]:
aaa

mpfr('2.0',100)

In [163]:
bbb

mpfr('0.30000000000000000000000000000016',100)

In [164]:
aaa+bbb

mpfr('2.2999999999999999999999999999994',100)

In [165]:
mpz(2)/mpz(3)

mpfr('0.66666666666666666666666666666693',100)

In [166]:
class Eliminate:
    """
    A data structure to record and identify
    whether a tree has been visited/pruned
    """
    def __init__(self, elim_dict = None, 
                 eliminated = None):
        self.elim_dict = {}
        self.eliminated = []
        
    def eliminate(self, prefix):
        self.elim_dict[tuple(sorted(prefix))] = 1
    
    def eliminate_children(self, dp):
        self.eliminated.append(dp)
        
    def is_duplicated(self, prefix):
        if tuple(sorted(prefix)) in self.elim_dict.keys():
            #print("001 Eliminated!")
            return True
        
        return False
        
    def is_eliminated(self, tree):
        prefix = tree.get_prefix()
        for i in range(len(self.eliminated)):
            if all([r in prefix for r in self.eliminated[i]]):
                idx = [prefix.index(r) for r in self.eliminated[i]]
                splitleaf_list = tree.get_splitleaf()
                splitleaf_array = np.array(splitleaf_list)
                if len(splitleaf_list)<=0 or splitleaf_array[:,idx].sum()<=0:
                    #print("002 Eliminated!")
                    #print("prefix::", tree.get_prefix())
                    #print(splitleaf_list)
                    return True
        
        return False
    
    def get_elim_dict(self):
        return self.elim_dict

In [175]:
def algorithm1_multi(x, y, lamb = LAMB):
    """
    A brute forece implementation of Algorithm1
    ## multiple copies of tree
    ## mark which leaf to split
    """
    
    d_c = None
    R_c = 1

    nrule = x.shape[1]
    ndata = len(y)
    print("nrule:", nrule)
    print("ndata:", ndata)

    # initialize the queue to include all trees of just one split
    #queue = [CacheTree(prefix = ((-r,),(r,)), x = x, y = y) for r in range(1, nrule+1)]

    queue = []
    for r in range(1, nrule+1):
        tree0 = CacheTree(prefix = ((-r,),(r,)), x = x, y = y)
        "heapq.heappush(queue, (tree0.get_curiosity(),tree0))"
        queue.append(tree0)
        R = tree0.get_lbound()[0]+(tree0.get_ncc()[0])/len(y)#Risk(tree,x,y,lamb)
        if R<R_c:
            d_c = tree0.get_prefix()
            R_c = R
    
    E = Eliminate()

    while (queue):
        tree = queue.pop(0)
        "(curio, tree)=heapq.heappop(queue)"
        d = tree.get_prefix()
        #print("E:::",E.get_elim_dict())
        if E.is_eliminated(tree) or E.is_duplicated(d):
            continue
        else:
            E.eliminate(d)

        #print("d",d)
        #print(tree.get_splitleaf())
        #print("tree.get_lbound()",tree.get_lbound())
        #print("tree.get_ncc()",tree.get_ncc())
        #print("d split:", tree.get_splitleaf())
        #print("R",tree.get_lbound()[0]+(tree.get_ncc()[0])/len(y))
        #print("R_c",R_c)
        #print(tree.get_ncc())
        #print(tree.get_cap())
        #print(tree.get_lbound())
        
        for i in range(len(d)):
            if tree.get_deadleaf()[i]==1:
                continue
            
            #(Lower bound on antecedent support)
            if tree.get_cap()[i]/ndata/2 < lamb:
                tree.set_deadleaf(i)
                #print("==============dead==============",i)
                continue
            
            spl = tree.get_splitleaf()
            split_next = spl[1:]
            
            if spl[0][i]==0:
                continue
            
            #print("splitleaf():",spl)
            #if len(split_next)>0:
            #    print("split_next[k][:i]",split_next[0][:i])
            #    print("split_next[k][i]",split_next[0][i])
            #    print("split_next[k][i+1:]",split_next[0][i+1:])
            splitleaf_list = [split_next[k][:i]+split_next[k][i+1:]+split_next[k][i:i+1]*2
                              for k in range(len(split_next))]
            
            d0 = d[i]
            dp = d[:i]+d[i+1:]
            #print("bound", bound(tree,x,y,i))
            #print("R_c", R_c)
            
            
            # Restrict the depth of the tree
            MAXDEPTH = 4
            if len(d0)>=MAXDEPTH:
                continue
            
            
            #if bound(tree,x,y,i,lamb)<R_c:
            lb = tree.get_lbound()[i]
            b0 = tree.get_B0()[i]
            if lb+b0+100>=R_c or lb>=R_c:
            #    pass#print("hello")
            #else:
            #    pass
            #if True:
                #print("===CASE 3===")
                E.eliminate_children(dp)
                for j in range(1, nrule+1):
                    if (j not in d0)and(-j not in d0):
                        l1 = d0+(-j,)
                        l2 = d0+(j,)
                        t = dp+(l1, l2)
                        #print("t",t)
                        
                        if E.is_duplicated(t):
                            continue
                        
                        pred_l, cap_l, incorr_l, B0_l = calcul((l1,l2),x,y)
                        loss_l1 = (incorr_l[0])/len(y)
                        loss_l2 = (incorr_l[1])/len(y)
                        loss_d0 = tree.get_ncc()[i]/len(y)
                        delta = loss_l1+loss_l2-loss_d0+lamb
                        old_lbound = tree.get_lbound()[:i]+tree.get_lbound()[i+1:]
                        new_lbound = [b+delta for b in old_lbound]+[tree.get_lbound()[i]+loss_l2+lamb,tree.get_lbound()[i]+loss_l1+lamb]
                        
                        
                        splitleaf1 = [1]*(len(t)) 
                        splitleaf2 = [0]*(len(t)-2)+[1,1] #next round split l1/l2
                        splitleaf3 = [1]*(len(t)-2)+[0,0]
                        
                        splitleaf_array = np.array(splitleaf_list)
                        
                        #(Lower bound on accurate antecedent support)
                        a_l = (sum(cap_l)-sum(incorr_l))/ndata - sum(cap_l)/ndata/2
                        if a_l < lamb:
                            #print("===CASE 4===")
                            sl = splitleaf_list+[splitleaf2]
                            E.eliminate_children((l1,l2))
                            ########### split l1/l2??????????????????????????????????????
                            if len(splitleaf_list)<=0 or splitleaf_array.sum()-splitleaf_array[:,i].sum()<=0:
                                sl = splitleaf_list+[splitleaf2]+[splitleaf3]
                        else:
                            if len(splitleaf_list)>0 and splitleaf_array.sum()-splitleaf_array[:,i].sum()>0:
                                sl = splitleaf_list+[splitleaf1]
                            else:
                                sl = splitleaf_list+[splitleaf3]
                        
                        tree_new = CacheTree(x = x, y = y, prefix = t,
                                             prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                                             num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                                             num_captured_incorrect = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+incorr_l,
                                             deadleaf = tree.get_deadleaf()[:i]+tree.get_deadleaf()[i+1:]+[0,0],
                                             splitleaf = sl,
                                             lbound = new_lbound,
                                             B0 = tree.get_B0()[:i]+tree.get_B0()[i+1:]+B0_l
                                            )
                        #print("tree_new:",tree_new.get_prefix())
                        #print("tree_new:",tree_new.get_splitleaf())
                        
                        if (not E.is_eliminated(tree_new)):
                            #pred_l, cap_l, corr_l = calcul((l1,l2),x,y)
                            queue.append(tree_new)
                            "heapq.heappush(queue, (tree_new.get_curiosity(),tree_new))"
                            R = tree_new.get_lbound()[0]+(tree_new.get_ncc()[0])/len(y)#Risk(tree,x,y,lamb)
                            if R<R_c:
                                d_c = t
                                R_c = R
                            #print("push::", t)
                            #print("222:",tree_new.get_splitleaf())
                #print("Children Elim")
                        
            else:
                
                
                #print("===CASE 1===")
                for j in range(1, nrule+1):
                    if (j not in d0)and(-j not in d0):
                        l1 = d0+(-j,)
                        l2 = d0+(j,)
                        t = dp+(l1, l2)
                        #print("t",t)
                        
                        if E.is_duplicated(t):
                            continue
                        
                        pred_l, cap_l, incorr_l, B0_l = calcul((l1,l2),x,y)
                        loss_l1 = (incorr_l[0])/len(y)
                        loss_l2 = (incorr_l[1])/len(y)
                        loss_d0 = tree.get_ncc()[i]/len(y)
                        delta = loss_l1+loss_l2-loss_d0+lamb
                        old_lbound = tree.get_lbound()[:i]+tree.get_lbound()[i+1:]
                        new_lbound = [b+delta for b in old_lbound]+[tree.get_lbound()[i]+loss_l2+lamb,tree.get_lbound()[i]+loss_l1+lamb]
                        
                        splitleaf = [1]*len(t)
                        
                        #(Lower bound on accurate antecedent support)
                        a_l = (sum(cap_l)-sum(incorr_l))/ndata - sum(cap_l)/ndata/2
                        if a_l < lamb:
                            #print("===CASE 2===")
                            splitleaf = [0]*(len(t)-2)+[1,1] #next round split l1/l2
                            E.eliminate_children((l1,l2))
                            ########### split l1/l2??????????????????????????????????????
                        
                        #if tree.get_tag()==True:
                        #    coma = [co==0 for co in tree.get_comaleaf()]
                        #    comaleaf = coma[:i]+coma[i+1:]+[1,1]
                        
                        sl = splitleaf_list+[splitleaf]
                        #print("here", sl)
                        tree_new = CacheTree(x = x, y = y, prefix = t,
                                             prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                                             num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                                             num_captured_incorrect = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+incorr_l,
                                             deadleaf = tree.get_deadleaf()[:i]+tree.get_deadleaf()[i+1:]+[0,0],
                                             splitleaf = sl,
                                             lbound = new_lbound,
                                             B0 = tree.get_B0()[:i]+tree.get_B0()[i+1:]+B0_l
                                            )
                        #print("tree_new:",tree_new.get_prefix())
                        #print("tree_new:",tree_new.get_splitleaf())
                        if (not E.is_eliminated(tree_new)):
                            #pred_l, cap_l, corr_l = calcul((l1,l2),x,y)
                            queue.append(tree_new)
                            "heapq.heappush(queue, (tree_new.get_curiosity(),tree_new))"
                            R = tree_new.get_lbound()[0]+(tree_new.get_ncc()[0])/len(y)#Risk(tree,x,y,lamb)
                            if R<R_c:
                                d_c = t
                                R_c = R
                            #print("push::", t)
                            #print("111",tree_new.get_splitleaf())

    print("d_c", d_c)
    print("R_c", R_c)

#### for 3 rules, 25 data, lambda = 0.04
algorithm  algorithm1  algorithm1_2splits  algorithm1_nobound
time     20.7ms    51.9ms          217ms

In [168]:
rule_idx = sample(range(13),13)

In [177]:
%%time
# algorithm1_2splits, 4 rules, all data, lambda = 0.0025############

algorithm1_multi(x[:,1:],y)

nrule: 4
ndata: 6907
d_c ((1,), (-1, -4), (-1, 4))
R_c 0.3764601129289127
CPU times: user 42.9 s, sys: 16 ms, total: 42.9 s
Wall time: 43 s


In [176]:
%%time
# algorithm1_2splits, 3 rules, all data, lambda = 0.0025############

algorithm1_multi(x[:,2:],y)

nrule: 3
ndata: 6907
d_c ((-3,), (3,))
R_c 0.3810829593166353
CPU times: user 584 ms, sys: 4 ms, total: 588 ms
Wall time: 586 ms


In [ ]:
%%time
# algorithm1_2splits, 5 rules, all data, lambda = 0.04

algorithm1_multi(x,y)

nrule: 5
ndata: 6907
